# Assignment 2

# Formalia:

Please read the [assignment overview page](https://laura.alessandretti.com/comsocsci2023/assignments.html) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment. 

_If you fail to follow these simple instructions, it will negatively impact your grade!_

**Due date and time**: The assignment is due on Mar 28th at 23:59. Hand in your Jupyter notebook file (with extension `.ipynb`) via DTU Learn _(Assignment, Assignment 2)_. 

Remember to include in the first cell of your notebook:
* the link to your group's Git repository (if you don't have a shared Git repository, it's fine. Remember to do it next time)
* group members' contributions

# Part 1: Mixing Patterns and Assortativity

Let's dig in and try to understand more about the network of Computational Social Scientists using more advanced features. If your network has more than one component, just work on the giant connected component (GCC) in the exercises below. For the exercises below, you shall use the network you built in [Week 4](https://github.com/lalessan/comsocsci2023/blob/main/lectures/Week4.ipynb).


> **Exercise 1: Mixing Patterns and Assortativity.** 
>
> * For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.
> * Create a new graph, with the same nodes and edges, but where the association between nodes and field is shuffled. Compute the measure above for this randomized graph.
> * Repeat the point above 100 times (at least). Plot the distribution of the values obtained and compare it with the value you have found for the real graph. Is the chance to connect to a member of the same field significantly higher than it would be by chance?
> * Compute the assortativity coefficient with respect to author's field. How do you interpret the value you obtain? (__Hint__: See [this paper](https://nbviewer.org/github/suneman/socialgraphs2019/blob/master/lectures/Week5.ipynb), eq (2)). **Important**: here I do not want you to use the NetworkX implementation, but rather to implement your own version of the measure.    
> * Is the graph assortative with respect to the degree? (e.g. do high-degree scientists tend to link to other high-degree scientists, and low-degree scientists to other low-degree scientists?). Provide an interpretation of your answer.
> * _Optional:_ Estimate the gender of each author from their name, using the [World Gender Name Dictionary](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MSEGSJ). Repeat the analysis above to study the assortativity of the network by gender rather than by field. What do you observe?


In [153]:
import json
import numpy as np
import netwulf
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import random

In [149]:
with open('/work/css_graph.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
    G = nx.node_link_graph(json_object)

largest_connected_component = max(nx.connected_components(G), key=len)

G = G.subgraph(largest_connected_component).copy()

print(G)

Graph with 28586 nodes and 85462 edges


For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.

In [177]:
nodes_adjacencies_list = [(n, nbrdict) for n, nbrdict in G.adjacency()]
node_fiels_dict = nx.get_node_attributes(G,'field')
node_adj_same_field_frac_list = []

# goes over all nodes and its neighbor's calculates the fractions of neighbors in the same field
def neighbor_same_field_frac(nodes_adjacencies_list, node_fiels_dict):
    for node_data in nodes_adjacencies_list:
        node_field = node_fiels_dict.get(node_data[0])
        
        neighbor_fields = [node_fiels_dict.get(key) for key in node_data[1].keys()]
        same_field_list = [1 if node_field == neighbor_field else 0 for neighbor_field in neighbor_fields]

        node_adj_same_field_frac_list.append(sum(same_field_list)/len(same_field_list))
        
        
    return np.mean(node_adj_same_field_frac_list)

node_adj_same_field_frac = neighbor_same_field_frac(nodes_adjacencies_list, node_fiels_dict)
print(f"node_adj_same_field_frac_list: {node_adj_same_field_frac}")

node_adj_same_field_frac_list: 0.488684929891156


In [185]:
def shuffle_node_fiels_dict(node_fiels_dict):
    uniqe_top_fields = list(np.unique(list(node_fiels_dict.values())))

    for key, value in node_fiels_dict.items(): 
        node_fiels_dict[key] = random.choice(uniqe_top_fields)
    return node_fiels_dict


node_fiels_dict = shuffle_node_fiels_dict(node_fiels_dict)

n = 100
neighbor_same_field_frac_shuffled_list = []
for i in range(n):
    node_fiels_dict = shuffle_node_fiels_dict(node_fiels_dict)
    neighbor_same_field_frac_shuffled_list.append(neighbor_same_field_frac(nodes_adjacencies_list, node_fiels_dict))

[0.427193832818435,
 0.42718365671304526,
 0.42717574482793014,
 0.42716607296978104,
 0.4271577681310996,
 0.4271530540938808,
 0.42714511090287477,
 0.42713864856818357,
 0.42713296580554283,
 0.4271226666090698,
 0.42711820286778984,
 0.42711659398478596,
 0.42711210515765946,
 0.4271025853113657,
 0.42709169490588617,
 0.4270881566044306,
 0.4270808727548975,
 0.4270752036350066,
 0.42707063788668864,
 0.42706328780702224,
 0.4270614166285447,
 0.4270576988496245,
 0.42705451713148185,
 0.4270475391837685,
 0.4270412439140803,
 0.4270391334849471,
 0.4270337003293905,
 0.4270322763560173,
 0.4270251170192848,
 0.4270186867013296,
 0.42701526212475216,
 0.4270124481095687,
 0.42700726886172996,
 0.4270039955157448,
 0.42700014298742806,
 0.4269963315992701,
 0.4269936066617715,
 0.4269900044478404,
 0.42698768231637957,
 0.42698236987071336,
 0.4269850464322753,
 0.42698250132554033,
 0.42697361150843827,
 0.4269702650285567,
 0.4269672201885769,
 0.426966575547275,
 0.4269617278692

# Part 2: Communities

> __Exercise 2: Zachary's karate club__: In this exercise, we will work on Zarachy's karate club graph (refer to the Introduction of Chapter 9). The dataset is available in NetworkX, by calling the function [karate_club_graph](https://networkx.org/documentation/stable//auto_examples/graph/plot_karate_club.html) 
>
> 1. Visualize the graph using [netwulf](https://netwulf.readthedocs.io/en/latest/). Set the color of each node based on the club split (the information is stored as a node attribute). My version of the visualization is below.
>
> 2. Write a function to compute the __modularity__ of a graph partitioning (use **equation 9.12** in the book). The function should take a networkX Graph and a partitioning as inputs and return the modularity.
> 3. Explain in your own words the concept of _modularity_. 
> 4. Compute the modularity of the Karate club split partitioning using the function you just wrote. Note: the Karate club split partitioning is avilable as a [node attribute](https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.classes.function.get_node_attributes.html), called _"club"_.
> 5. We will now perform a small randomization experiment to assess if the modularity you just computed is statitically different from $0$. To do so, we will implement the _double edge swap_ algorithm. The _double edge swap_ algorithm is quite old... it was implemented in 1891 (!) by Danish mathematician Julius Petersen(https://en.wikipedia.org/wiki/Julius_Petersen). Given a network G, this algorithm creates a new network, such that each node has exactly the same degree as in the original network, but different connections. Here is how the algorithm works.
>       * __a.__ Create an identical copy of your original network.
>       * __b.__ Consider two edges in your new network (u,v) and (x,y), such that u!=v and v!=x.
>       * __c.__ If none of edges (u,y) and (x,v) exists already, add them to the network and remove edges (u,v) and (x,y).
>       * Repeat steps __b.__ and __c.__ to achieve at least N swaps (I suggest N to be larger than the number of edges).
> 6. Double check that your algorithm works well, by showing that the degree of nodes in the original network and the new 'randomized' version of the network are the same.
> 7. Create $1000$ randomized version of the Karate Club network using the _double edge swap_ algorithm you wrote in step 5. For each of them, compute the modularity of the "club" split and store it in a list.
> 8. Compute the average and standard deviation of the modularity for the random network.
> 9. Plot the distribution of the "random" modularity. Plot the actual modularity of the club split as a vertical line (use [axvline](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axvline.html)). 
> 10. Comment on the figure. Is the club split a good partitioning? Why do you think I asked you to perform a randomization experiment? What is the reason why we preserved the nodes degree?
> 11.  Use [the Python Louvain-algorithm implementation](https://anaconda.org/auto/python-louvain) to find communities in this graph. Report the value of modularity found by the algorithm. Is it higher or lower than what you found above for the club split? What does this comparison reveal?
> 12.  Compare the communities found by the Louvain algorithm with the club split partitioning by creating a matrix **_D_** with dimension (2 times _A_), where _A_ is the number of communities found by Louvain. We set entry _D_(_i_,_j_) to be the number of nodes that community _i_ has in common with group split _j_. The matrix **_D_** is what we call a [**confusion matrix**](https://en.wikipedia.org/wiki/Confusion_matrix). Use the confusion matrix to explain how well the communities you've detected correspond to the club split partitioning.

1. Visualize the graph using netwulf. Set the color of each node based on the club split (the information is stored as a node attribute). My version of the visualization is below.

In [127]:
KCG = nx.karate_club_graph()
community_header = np.unique([x for x in nx.get_node_attributes(KCG,'club').values()])
print(community_header)
## There's two different clubs, so let's color them using two colors:
for k, v in KCG.nodes(data=True):
    if v['club'] == 'Mr. Hi':
        v['color'] = 'red'
    else:
        v['color'] = 'blue'

#netwulf.visualize(KCG) ## Netwulf doesn't run on deepnote, but a picture is included of the one running locally

['Mr. Hi' 'Officer']


![](kcg.png)

2. Write a function to compute the modularity of a graph partitioning (use equation 9.12 in the book). The function should take a networkX Graph and a partitioning as inputs and return the modularity.

In [170]:
def compute_modularity(G, partition):
    modularity = 0
    community_header = np.unique([x for x in partition.values()]) ## List of unique communities
    for community in community_header:
        community_indices = [i for i, j in enumerate(partition.values()) if j == community] ## Find indices of nodes in community
        Sg = G.subgraph(community_indices) ## Subgraph for community
        degreesum = sum([y for x, y in Sg.degree]) ## Sum of degrees in subgraph
        modularity += len(Sg.edges)/len(G.edges) - ((degreesum/(2*len(G.edges)))**2) ## Inside of sum in eq (9.12)
    return modularity


print(compute_modularity(KCG, nx.get_node_attributes(KCG,'club')))

0.48931623931623935


3. Explain in your own words the concept of modularity.

SyntaxError: invalid syntax (2637169335.py, line 1)

> __Exercise 3__: Community detection on the network of Computational Social Scientists. 
>
> * Consider the network you built in [Week 4](https://github.com/lalessan/comsocsci2023/blob/main/lectures/Week4.ipynb).
> * Use [the Python Louvain-algorithm implementation](https://anaconda.org/auto/python-louvain) to find communities. How many communities do you find? What are their sizes? Report the value of modularity found by the algorithm. Is the modularity significantly different than 0? 
> * If you are curious, you can also try the *Infomap* algorithm. Go to [this page]. (https://mapequation.github.io/infomap/python/). It's harder to install, but a better community detection algorithm. You can read about it in [advanced topics 9B](http://networksciencebook.com/chapter/9#advanced-9b).
> * Visualize the network, using netwulf (see Week 5). This time assign each node a different color based on their _community_. Describe the structure you observe.
> * Make sure you save the assignment of authors to communities. 


# Part 3: TF-IDF.


> __Exercise 4: TF-IDF and the Computational Social Science communities.__ The goal for this exercise is to find the words charachterizing each of the communities of Computational Social Scientists.
> What you ned for this exercise: 
>    * The assignment of each author to their network community, and the degree of each author (Week 6, Exercise 4). This can be stored in a dataframe or in two dictionaries, as you prefer.  
>    * the tokenized _abstract_ dataframe (Week 7, Exercise 2)
>
> 1. First, check out [the wikipedia page for TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). Explain in your own words the point of TF-IDF. 
>   * What does TF stand for? 
>   * What does IDF stand for?
> 2. Now, we want to find out which words are important for each *community*, so we're going to create several ***large documents, one for each community***. Each document includes all the tokens of abstracts written by members of a given community. 
>   * Consider a community _c_
>   * Find all the abstracts of papers written by a member of community _c_.
>   * Create a long array that stores all the abstract tokens 
>   * Repeat for all the communities. 
> __Note:__ Here, to ensure your code is efficient, you shall exploit ``pandas`` builtin functions, such as [``groupby.apply``](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.apply.html) or [``explode``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html).
> 3. Now, we're ready to calculate the TF for each word. Use the method of your choice to find the top 5 terms within the __top 5 communities__ (by number of authors). 
>   * Describe similarities and differences between the communities.
>   * Why aren't the TFs not necessarily a good description of the communities?
>   * Next, we calculate IDF for every word. 
>   * What base logarithm did you use? Is that important?
> 4. We're ready to calculate TF-IDF. Do that for the __top 9 communities__ (by number of authors). Then for each community: 
>   * List the 10 top TF words 
>   * List the 10 top TF-IDF words
>   * List the top 3 authors (by degree)
>   * Are these 10 words more descriptive of the community? If yes, what is it about IDF that makes the words more informative?


> __Exercise 5: The Wordcloud__. It's time to visualize our results!
> * Install the [`WordCloud`](https://pypi.org/project/wordcloud/) module. 
> * Now, create word-cloud for each community. Feel free to make it as fancy or non-fancy as you like.
> * Make sure that, together with the word cloud, you print the names of the top three authors in each community (see my plot above for inspiration). 
> * Comment on your results. What can you conclude on the different sub-communities in Computational Social Science? 
> * Look up online the top author in each community. In light of your search, do your results make sense?

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2916efa0-c445-43ca-8dcd-0a1bcdbbd016' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>